***
__GEECol classes: GEECol.getcollection(...) method using reference collection__

- using reference collection to obtain congruent (raster) images
- illustrate s2 10m vs 20m roi alignment problems

***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils
import geeproduct


__Proba V 333m with reference S2 10m__

In [ ]:
#
# reference
#
refgeecol            = geeproduct.GEECol_s2ndvi()
refroipixelsdiameter = 32
#
# target
#
trggeecol            = geeproduct.GEECol_pv333ndvi()
trgroipixelsdiameter = 3 # demo. normally we'd use maximum 2 = int(32*10 / 330) + 2 
#
# parameters
#
eepoint              = geeutils.tapspoint
eedatefrom           = geeutils.fleecycloudsday
eedatetill           = eedatefrom.advance(1, 'week')
verbose              = False
#
# get target collection using refcollection and refroipixelsdiameter
#
trggeecollection = trggeecol.getcollection(
    eedatefrom, eedatetill, eepoint, trgroipixelsdiameter, 
    refcollection=refgeecol, refroipixelsdiameter=refroipixelsdiameter, verbose=verbose)

szdescription    = trggeecollection.get('gee_description').getInfo()
eerefroi         = ee.Geometry(trggeecollection.get('gee_refroi'))
eecenterpoint    = ee.Geometry(trggeecollection.get('gee_centerpoint'))
eefirsttrgimage  = trggeecollection.first()
szdate           = eefirsttrgimage.get('gee_date').getInfo()
#
# beware: these are only available as (latched) debug info
#
eerefimage       = trggeecol._eerefimage                    # the image selected as reference during getcollection
eesrcimage       = trggeecol._eenatimagecollection.first()  # the first image in the target collection in its native form


#
# visualization
#

# estimate range in roi environment for visualization
xenv = geeutils.squarerasterboundsroi(eecenterpoint, trgroipixelsdiameter*5, eefirsttrgimage)
xtrgmin = eefirsttrgimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xtrgmax = eefirsttrgimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()
xrefmin = eerefimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xrefmax = eerefimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()
print(szdescription, szdate, "min:", xtrgmin, "max:", xtrgmax)

# apply masks for visualization
src = geeutils.maskinsidegeometry(eerefroi,  eesrcimage, 220)
ref = geeutils.maskinsidegeometry(eerefroi,  geeutils.maskoutsidegeometry(eerefroi, eerefimage, 180), 80)
tgt = geeutils.maskoutsidegeometry(eerefroi, eefirsttrgimage, 40)

# show it
map = geemap.Map(height='600px')
map.centerObject(eecenterpoint, 16)

map.addLayer(ee.Image(0), {'palette':'#ffff00'})                 # yellow background
map.addLayer(src, {'min':xtrgmin, 'max':xtrgmax}, 'eesrcimage')  # target - native (has same date as target)
map.addLayer(ref, {'min':xrefmin, 'max':xrefmax}, 'eerefimage')  # reference - can have other date
map.addLayer(tgt, {'min':xtrgmin, 'max':xtrgmax}, szdescription) # target congruent with reference

map.addLayer(geeutils.outlinegeometryimage(eerefroi,       0, 2), {'palette':'#ff0000'}, 'eerefroi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,       10, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eecenterpoint, 10, 2), {'palette':'#ff0000'}, 'eeroicenterpoint')
map


__Sentinel 1 VV/VH bands - ASCENDING/DESCENDING - sigma0/gamma0 with reference S2 10m__

In [ ]:
refgeecol            = geeproduct.GEECol_s2ndvi()
refroipixelsdiameter = 8

#trggeecol        = geeproduct.GEECol_s1sigma0('VV', 'DESCENDING')
#trggeecol        = geeproduct.GEECol_s1sigma0('VH', 'DESCENDING')
#trggeecol        = geeproduct.GEECol_s1sigma0('VV', 'ASCENDING')
#trggeecol        = geeproduct.GEECol_s1sigma0('VH', 'ASCENDING')
#trggeecol        = geeproduct.GEECol_s1gamma0('VV', 'DESCENDING')
#trggeecol        = geeproduct.GEECol_s1gamma0('VH', 'DESCENDING')
#trggeecol        = geeproduct.GEECol_s1gamma0('VV', 'ASCENDING')
trggeecol        = geeproduct.GEECol_s1gamma0('VH', 'ASCENDING')

trgroipixelsdiameter = 8


eepoint              = geeutils.tapspoint
eedatefrom           = geeutils.fleecycloudsday
eedatetill           = eedatefrom.advance(1, 'week')
verbose              = False
#
#
#
trggeecollection = trggeecol.getcollection(
    eedatefrom, eedatetill, eepoint, trgroipixelsdiameter, 
    refcollection=refgeecol, refroipixelsdiameter=refroipixelsdiameter, verbose=verbose)

szdescription    = trggeecollection.get('gee_description').getInfo()
eerefroi         = ee.Geometry(trggeecollection.get('gee_refroi'))
eecenterpoint    = ee.Geometry(trggeecollection.get('gee_centerpoint'))
eefirsttrgimage  = trggeecollection.first()
szdate           = eefirsttrgimage.get('gee_date').getInfo()

eerefimage       = trggeecol._eerefimage                    # the image selected as reference during getcollection
eesrcimage       = trggeecol._eenatimagecollection.first()  # the first image in the target collection in its native form


#
# visualization
#

# estimate range in roi environment for visualization
xenv = geeutils.squarerasterboundsroi(eecenterpoint, trgroipixelsdiameter*5, eefirsttrgimage)
xtrgmin = eefirsttrgimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xtrgmax = eefirsttrgimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()
xrefmin = eerefimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xrefmax = eerefimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()
print(szdescription, szdate, "min:", xtrgmin, "max:", xtrgmax)

# apply masks for visualization
src = geeutils.maskinsidegeometry(eerefroi,  eesrcimage, 60)
ref = geeutils.maskinsidegeometry(eerefroi,  geeutils.maskoutsidegeometry(eerefroi, eerefimage, 50), 30)
tgt = geeutils.maskoutsidegeometry(eerefroi, eefirsttrgimage, 20)

# show it
map = geemap.Map(height='600px')
map.centerObject(eecenterpoint, 18)

map.addLayer(ee.Image(0), {'palette':'#ffff00'})                 # yellow background
map.addLayer(src, {'min':xtrgmin, 'max':xtrgmax}, 'eesrcimage')  # target - native (has same date as target)
map.addLayer(ref, {'min':xrefmin, 'max':xrefmax}, 'eerefimage')  # reference - can have other date
map.addLayer(tgt, {'min':xtrgmin, 'max':xtrgmax}, szdescription) # target congruent with reference

map.addLayer(geeutils.outlinegeometryimage(eerefroi,       0, 2), {'palette':'#ff0000'}, 'eerefroi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,       10, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eecenterpoint, 10, 2), {'palette':'#ff0000'}, 'eeroicenterpoint')
map

__Sentinel 1 VV/VH bands - RVI: 4VH/(VV+VH) with reference S2 10m__

In [ ]:
refgeecol            = geeproduct.GEECol_s2ndvi()
refroipixelsdiameter = 14

#trggeecol            = geeproduct.GEECol_s1rvi('DESCENDING')
trggeecol            = geeproduct.GEECol_s1rvi('ASCENDING')

trgroipixelsdiameter = 7 # demo - normally we'd use same as ~refroipixelsdiameter since they have same resolution


eepoint              = geeutils.tapspoint
eedatefrom           = geeutils.fleecycloudsday
eedatetill           = eedatefrom.advance(1, 'week')
verbose              = False
#
#
#
trggeecollection = trggeecol.getcollection(
    eedatefrom, eedatetill, eepoint, trgroipixelsdiameter, 
    refcollection=refgeecol, refroipixelsdiameter=refroipixelsdiameter, verbose=verbose)

szdescription    = trggeecollection.get('gee_description').getInfo()
eerefroi         = ee.Geometry(trggeecollection.get('gee_refroi'))
eecenterpoint    = ee.Geometry(trggeecollection.get('gee_centerpoint'))
eefirsttrgimage  = trggeecollection.first()
szdate           = eefirsttrgimage.get('gee_date').getInfo()

eerefimage       = trggeecol._eerefimage                    # the image selected as reference during getcollection
eesrcimage       = trggeecol._eenatimagecollection.first()  # the first image in the target collection in its native form


#
# visualization
#

# estimate range in roi environment for visualization
xenv = geeutils.squarerasterboundsroi(eecenterpoint, trgroipixelsdiameter*5, eefirsttrgimage)
xtrgmin = eefirsttrgimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xtrgmax = eefirsttrgimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()
xrefmin = eerefimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xrefmax = eerefimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()
print(szdescription, szdate, "min:", xtrgmin, "max:", xtrgmax)

# apply masks for visualization
src = geeutils.maskinsidegeometry(eerefroi,  eesrcimage, 60)
ref = geeutils.maskinsidegeometry(eerefroi,  geeutils.maskoutsidegeometry(eerefroi, eerefimage, 50), 30)
tgt = geeutils.maskoutsidegeometry(eerefroi, eefirsttrgimage, 20)

# show it
map = geemap.Map(height='600px')
map.centerObject(eecenterpoint, 17)

map.addLayer(ee.Image(0), {'palette':'#ffff00'})                 # yellow background
map.addLayer(src, {'min':xtrgmin, 'max':xtrgmax}, 'eesrcimage')  # target - native (has same date as target)
map.addLayer(ref, {'min':xrefmin, 'max':xrefmax}, 'eerefimage')  # reference - can have other date
map.addLayer(tgt, {'min':xtrgmin, 'max':xtrgmax}, szdescription) # target congruent with reference

map.addLayer(geeutils.outlinegeometryimage(eerefroi,       0, 2), {'palette':'#ff0000'}, 'eerefroi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,       10, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eecenterpoint, 10, 2), {'palette':'#ff0000'}, 'eeroicenterpoint')
map

__Sentinel 2 SCL bands with reference S2 10m__

- where problems live:
- native scl and resampled scl will both line up with s2 10m
- but resampled scl can be either the original scl, or median of 4 scl pixels
- depending on the eepoint location -  roi center is aligned on s2 10m

In [ ]:
refgeecol            = geeproduct.GEECol_s2ndvi()
refroipixelsdiameter = 12

trggeecol            = geeproduct.GEECol_s2scl()
trgroipixelsdiameter = 6


eepoint              = geeutils.bobspoint
#eepoint              = geeutils.hogerielenpoint
#eepoint              = geeutils.tapspoint
#eepoint              = geeutils.pannekoekpoint
#eepoint              = geeutils.tennvenlopoint
#eepoint              = geeutils.half31UESpoint
#eepoint              = geeutils.hoogeheydepoint
#eepoint              = geeutils.pastacosipoint

eedatefrom           = geeutils.fleecycloudsday
eedatetill           = eedatefrom.advance(1, 'week')
verbose              = False
#
#
#
trggeecollection = trggeecol.getcollection(
    eedatefrom, eedatetill, eepoint, trgroipixelsdiameter, 
    refcollection=refgeecol, refroipixelsdiameter=refroipixelsdiameter, verbose=verbose)

szdescription    = trggeecollection.get('gee_description').getInfo()
eerefroi         = ee.Geometry(trggeecollection.get('gee_refroi'))
eecenterpoint    = ee.Geometry(trggeecollection.get('gee_centerpoint'))
eefirsttrgimage  = trggeecollection.first()
szdate           = eefirsttrgimage.get('gee_date').getInfo()

eerefimage       = trggeecol._eerefimage                    # the image selected as reference during getcollection
eesrcimage       = trggeecol._eenatimagecollection.first()  # the first image in the target collection in its native form


#
# visualization
#

# estimate range in roi environment for visualization
xenv = geeutils.squarerasterboundsroi(eecenterpoint, trgroipixelsdiameter*5, eefirsttrgimage)
xtrgmin = eefirsttrgimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xtrgmax = eefirsttrgimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()
xrefmin = eerefimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xrefmax = eerefimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()
print(szdescription, szdate, "min:", xtrgmin, "max:", xtrgmax)

# apply masks for visualization
src = geeutils.maskinsidegeometry(eerefroi,  eesrcimage, 8)
ref = geeutils.maskinsidegeometry(eerefroi,  geeutils.maskoutsidegeometry(eerefroi, eerefimage, 6), 2)
tgt = geeutils.maskoutsidegeometry(eerefroi, eefirsttrgimage, 0)

# show it
map = geemap.Map(height='600px')
map.centerObject(eecenterpoint, 18)

map.addLayer(ee.Image(0), {'palette':'#ffffff'})                 # white background
map.addLayer(src, geeutils.s2sclvisParams, 'eesrcimage')         # target - native (has same date as target)
map.addLayer(ref, {'min':xrefmin, 'max':xrefmax}, 'eerefimage')  # reference - can have other date
map.addLayer(tgt, geeutils.s2sclvisParams, szdescription)        # target congruent with reference

map.addLayer(geeutils.outlinegeometryimage(eerefroi,       0, 1), {'palette':'#ff0000'}, 'eerefroi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,        6, 1), {'palette':'#0000ff'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eecenterpoint,  6, 1), {'palette':'#ff0000'}, 'eeroicenterpoint')
map

__Sentinel 2 SCL ConvMask band with reference S2 10m__

In [ ]:
refgeecol            = geeproduct.GEECol_s2ndvi()
refroipixelsdiameter = 80

trggeecol            = geeproduct.GEECol_s2sclconvmask()

trgroipixelsdiameter = 40


eepoint              = geeutils.tapspoint
eedatefrom           = geeutils.fleecycloudsday
eedatetill           = eedatefrom.advance(1, 'week')
verbose              = False
#
#
#
trggeecollection = trggeecol.getcollection(
    eedatefrom, eedatetill, eepoint, trgroipixelsdiameter, 
    refcollection=refgeecol, refroipixelsdiameter=refroipixelsdiameter, verbose=verbose)

szdescription    = trggeecollection.get('gee_description').getInfo()
eerefroi         = ee.Geometry(trggeecollection.get('gee_refroi'))
eecenterpoint    = ee.Geometry(trggeecollection.get('gee_centerpoint'))
eefirsttrgimage  = trggeecollection.first()
szdate           = eefirsttrgimage.get('gee_date').getInfo()

eerefimage       = trggeecol._eerefimage                    # the image selected as reference during getcollection
eesrcimage       = trggeecol._eenatimagecollection.first()  # the first image in the target collection in its native form


#
# visualization
#

# estimate range in roi environment for visualization
xenv = geeutils.squarerasterboundsroi(eecenterpoint, trgroipixelsdiameter*5, eefirsttrgimage)
xtrgmin = 0
xtrgmax = 1
xrefmin = eerefimage.clip(xenv).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xrefmax = eerefimage.clip(xenv).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()
print(szdescription, szdate, "min:", xtrgmin, "max:", xtrgmax)

# clear sky transparant
eesrcimage = eesrcimage.updateMask(eesrcimage.eq(1)) # clear sky transparant
eetrgimage = eefirsttrgimage.updateMask(eefirsttrgimage.eq(1))

#
# apply masks for visualization
src = geeutils.maskinsidegeometry(eerefroi,  eesrcimage, 240)
ref = geeutils.maskinsidegeometry(eerefroi,  geeutils.maskoutsidegeometry(eerefroi, eerefimage, 140), 40)
tgt = geeutils.maskoutsidegeometry(eerefroi, eetrgimage, 20)

# use experimental s2rgb product (using 'TCI' bands) as background
#
eedate       = ee.Date(szdate)
eergbimage   = geeproduct.GEECol_s2rgb().collect(eecenterpoint, eedatefrom, eedatetill).first()

# show it
map = geemap.Map(height='600px')
map.centerObject(eecenterpoint, 14)

map.addLayer(eergbimage, {'min':1,  'max':255}, 'rgb')           # rgb background
map.addLayer(src, {'min':xtrgmin, 'max':xtrgmax}, 'eesrcimage')  # target - native (has same date as target)
map.addLayer(ref, {'min':xrefmin, 'max':xrefmax}, 'eerefimage')  # reference - can have other date
map.addLayer(tgt, {'min':xtrgmin, 'max':xtrgmax}, szdescription) # target congruent with reference

map.addLayer(geeutils.outlinegeometryimage(eerefroi,       0, 2), {'palette':'#ff0000'}, 'eerefroi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,       10, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eecenterpoint, 10, 2), {'palette':'#ff0000'}, 'eeroicenterpoint')
map